In [2]:
import numpy as np
from astropy.table import Table, vstack
import astropy
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import ascii
import glob
from jagb_func import custom_hist, simple_gaussian, GLOESS
import statistics
from matplotlib import gridspec
from astropy.coordinates import ICRS, Galactic, Distance, Angle, SkyCoord
from astropy import units as u
from astropy.io import fits
from scipy.optimize import leastsq
from scipy.special import erf
from scipy.optimize import curve_fit
from astropy.wcs import WCS
from scipy import stats


## Define Functions

In [3]:
def compute_rgc(ra,dec, glx_ctr=SkyCoord(10.68458*u.degree, 41.2691*u.degree, frame='icrs'), glx_PA=Angle('38d'), glx_incl=Angle('74d'), glx_dist=Distance(776, unit=u.kpc)):
    """
    Computes deprojected radial distance of a point in a galaxy. All parameters for M31 are from Gregersen+15
    This is modified code based off code by P. Parmby (https://gist.github.com/PBarmby/9355846)

    coord: coordinates of given point
    glx_ctr: galaxy center
    glx_PA: galaxy position angle
    glx_incl: galaxy inclination angle
    glx_dist: distance to galaxy
    """
    coord = SkyCoord(ra*u.degree, dec*u.degree, frame='icrs')
    
    sky_radius = glx_ctr.separation(coord)
    avg_dec = 0.5 * (glx_ctr.dec + coord.dec).radian
    
    x = (glx_ctr.ra - coord.ra) * np.cos(avg_dec)
    y = glx_ctr.dec - coord.dec
    
    phi = glx_PA - Angle('90d') + Angle(np.arctan(y.arcsec / x.arcsec), unit=u.rad)
    xp = (sky_radius * np.cos(phi.radian)).arcmin
    yp = (sky_radius * np.sin(phi.radian)).arcmin

    ypp = yp / np.cos(glx_incl.radian)
    obj_radius = np.sqrt(xp ** 2 + ypp ** 2)  # in arcmin
    obj_dist = Distance(Angle(obj_radius, unit=u.arcmin).radian * glx_dist,
            unit=glx_dist.unit)

    return obj_dist


## Compute the deprojected radius of each field
First, we will compute the average R.A. and Dec. of each field. Then, we can use the function compute_rgc to compute the radial distance to that field from the center of M31. 

In [ ]:
raw_phot_files = sorted(glob.glob('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/*')) # these are the fits files from the PHAT team website
order = [21,23,22,18,19,20] # order in which they are being read through glob

average_ra=[]
average_dec=[]
field_nom=[]
brick_nom=[]

for j in range(6): # loop over six bricks
    t =  Table.read(raw_phot_files[j])
    
    # good star cuts
    t=t[(t['f110w_gst']==True)&(t['f814w_gst']==True)&(t['f110w_vega']<90)&(t['f814w_vega']<90)]

    for i in range(18): # loop over 18 fields for each brick
        t_i = t[t['field']==i+1]

        average_ra.append(np.mean(t_i['ra']))
        average_dec.append( np.mean(t_i['dec']))
        field_nom.append(i+1)
        brick_nom.append(order[j])
        
distances = pd.DataFrame({'Central RA':average_ra, 'Central dec':average_dec, 'field_nom':field_nom,'brick_nom':brick_nom})

rgc_list=[]
for i in range(len(distances)):
    rgc_list.append(np.array(compute_rgc(ra=np.array(distances['Central RA'])[i], dec=np.array(distances['Central dec'])[i])))
distances['rgc'] = np.array(rgc_list).flatten()

# sorted distances dataframe now gives the average RGC for each field. 
sorted_distances = distances.sort_values('rgc')

# drop brick 22 field 8 because of earthlimb IR background
sorted_distances=sorted_distances.drop(sorted_distances[(sorted_distances['brick_nom']==22)&(sorted_distances['field_nom']==8)].index)


In [5]:
# show what dataframe looks like
distances = pd.read_csv('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/tmp/binned_distances.csv')    
distances.head()

,Unnamed: 0,Central RA,Central dec,field_nom,brick_nom,rgc
0,0,11.655650,42.208827,1,21,16.075355
1,1,11.616019,42.214559,2,21,15.975177
2,2,11.571171,42.219450,3,21,15.969295
3,3,11.526824,42.224715,4,21,16.102721
4,4,11.482915,42.229990,5,21,16.363578


# Next, we want to split the 107 fields into 10 radially increasing regions, each with approximately equal numbers of JAGB stars

### Read in PHAT photometry of bricks 18-23, and combine into 1 dataframe

In [7]:
# # apply good star cuts and remove stars with magnitudes of 99.999
t= Table.read('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/hlsp_phat_hst_wfc3-uvis-acs-wfc-wfc3-ir_12108-m31-b18_f275w-f336w-f475w-f814w-f110w-f160w_v2_st.fits')
b18=t[(t['f110w_gst']==True)&(t['f814w_gst']==True)&(t['f110w_vega']<90)&(t['f814w_vega']<90)]
b18['Brick']=np.ones(len(b18))*18

t = Table.read('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/hlsp_phat_hst_wfc3-uvis-acs-wfc-wfc3-ir_12110-m31-b19_f275w-f336w-f475w-f814w-f110w-f160w_v2_st.fits')
b19=t[(t['f110w_gst']==True)&(t['f814w_gst']==True)&(t['f110w_vega']<90)&(t['f814w_vega']<90)]
b19['Brick']=np.ones(len(b19))*19

t = Table.read('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/hlsp_phat_hst_wfc3-uvis-acs-wfc-wfc3-ir_12112-m31-b20_f275w-f336w-f475w-f814w-f110w-f160w_v2_st.fits')
b20=t[(t['f110w_gst']==True)&(t['f814w_gst']==True)&(t['f110w_vega']<90)&(t['f814w_vega']<90)]
b20['Brick']=np.ones(len(b20))*20

t = Table.read('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/hlsp_phat_hst_wfc3-uvis-acs-wfc-wfc3-ir_12055-m31-b21_f275w-f336w-f475w-f814w-f110w-f160w_v2_st.fits')
b21=t[(t['f110w_gst']==True)&(t['f814w_gst']==True)&(t['f110w_vega']<90)&(t['f814w_vega']<90)]
b21['Brick']=np.ones(len(b21))*21

t = Table.read('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/hlsp_phat_hst_wfc3-uvis-acs-wfc-wfc3-ir_12076-m31-b22_f275w-f336w-f475w-f814w-f110w-f160w_v2_st.fits')
b22=t[(t['f110w_gst']==True)&(t['f814w_gst']==True)&(t['f110w_vega']<90)&(t['f814w_vega']<90)]
b22.remove_rows(b22['field']==8) # remove field 8
b22['Brick']=np.ones(len(b22))*22

t = Table.read('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/hlsp_phat_hst_wfc3-uvis-acs-wfc-wfc3-ir_12070-m31-b23_f275w-f336w-f475w-f814w-f110w-f160w_v2_st.fits')
b23=t[(t['f110w_gst']==True)&(t['f814w_gst']==True)&(t['f110w_vega']<90)&(t['f814w_vega']<90)]
b23['Brick']=np.ones(len(b23))*23

t = vstack([b18, b19, b20, b21, b22, b23])

# jagb star color cuts and lower magnitude cut (to avoid MS stars)
jagb_stars = t[((t['f814w_vega']-t['f110w_vega'])<2.3)&((t['f814w_vega']-t['f110w_vega'])>2)&(t['f110w_vega']<22)]

print('No. of JAGB stars = '+str(len(jagb_stars)))


No. of JAGB stars = 3657


### Now seperate all the JAGB stars into 10 regions. Because there are 3657 JAGB stars, we want each region to have approximately 350 stars.

In [ ]:
no_JAGB = 347 # number of JAGB stars per region.

r1=Table(); r2 = Table(); r3 = Table(); r4 = Table(); r5 = Table();  r6 = Table();  r7 = Table();  r8 = Table();  r9 = Table();  r10 = Table(); 
length_r1 = 0; length_r2 = 0; length_r3 = 0;length_r4 = 0; length_r5 = 0;length_r6 = 0;length_r7 = 0;length_r8 = 0;length_r9 = 0;length_r10 = 0;
k=0
k_list=[]

average_rgc = []

for j in range(10):
    if j==0:
        r = r1
        length_r = length_r1
    if j==1:
        r = r2
        length_r = length_r2
        
    if j==2:
        r = r3
        length_r = length_r3
        
    if j==3:
        r = r4
        length_r = length_r4
    if j==4:
        r = r5
        length_r = length_r5
        
    if j==5:
        r = r6
        length_r = length_r6
    if j==6:
        r = r7
        length_r = length_r7
    if j==7:
        r = r8
        length_r = length_r8
    if j==8:
        r = r9
        length_r = length_r9
        
    if j==9:
        r = r10
        length_r = length_r10

    rgc_i = []
    
    for i in range(k, 107):
        field_nom_i = np.array(sorted_distances['field_nom'])[i]
        brick_nom_i = np.array(sorted_distances['brick_nom'])[i]
        jagb_stars_i = jagb_stars[(jagb_stars['Brick']==brick_nom_i)&(jagb_stars['field']==field_nom_i)]
        rgc_i.append(np.array(sorted_distances['rgc'])[i])
        
        r= vstack([r, jagb_stars_i])

        length_r+=len(jagb_stars_i)
        k+=1

        if length_r>no_JAGB:
            break
    average_rgc.append(np.mean(rgc_i))
    if j ==0:
        r1=r
    if j ==1:
        r2=r
    if j ==2:
        r3=r
    if j ==3:
        r4=r
    if j ==4:
        r5=r
    if j ==5:
        r6=r
    if j ==6:
        r7=r
    if j ==7:
        r8=r
    if j ==8:
        r9=r
    if j ==9:
        r10=r
    k_list.append(k)


#### We also will want to seperate all of the stars into the regions, not just the JAGB stars

In [ ]:
t1 = t[((t['Brick']==18)&(t['field']==18))|((t['Brick']==18)&(t['field']==12))|((t['Brick']==18)&(t['field']==6))
      |((t['Brick']==19)&(t['field']==16))|((t['Brick']==19)&(t['field']==17))|((t['Brick']==18)&(t['field']==17))]

t2 = t[((t['Brick']==19)&(t['field']==15))|((t['Brick']==18)&(t['field']==11))|((t['Brick']==19)&(t['field']==18))
      |((t['Brick']==18)&(t['field']==5))|((t['Brick']==19)&(t['field']==14))|((t['Brick']==19)&(t['field']==10))
      |((t['Brick']==19)&(t['field']==9))|((t['Brick']==19)&(t['field']==11))]

t3 = t[((t['Brick']==19)&(t['field']==8))|((t['Brick']==18)&(t['field']==16))|((t['Brick']==19)&(t['field']==13))
      |((t['Brick']==18)&(t['field']==10))|((t['Brick']==19)&(t['field']==12))|((t['Brick']==18)&(t['field']==4))
      |((t['Brick']==19)&(t['field']==7))|((t['Brick']==19)&(t['field']==3))]

t4 = t[((t['Brick']==19)&(t['field']==4))|((t['Brick']==19)&(t['field']==2))|((t['Brick']==19)&(t['field']==5))
      |((t['Brick']==20)&(t['field']==18))|((t['Brick']==19)&(t['field']==1))|((t['Brick']==20)&(t['field']==12))
      |((t['Brick']==18)&(t['field']==9))|((t['Brick']==18)&(t['field']==3))|((t['Brick']==19)&(t['field']==6))
      |((t['Brick']==18)&(t['field']==15))]

t5 = t[((t['Brick']==21)&(t['field']==16))|((t['Brick']==21)&(t['field']==15))|((t['Brick']==21)&(t['field']==17))
      |((t['Brick']==20)&(t['field']==6))|((t['Brick']==21)&(t['field']==14))|((t['Brick']==20)&(t['field']==17))
      |((t['Brick']==21)&(t['field']==18))|((t['Brick']==21)&(t['field']==13))|((t['Brick']==20)&(t['field']==11))
      |((t['Brick']==21)&(t['field']==9))]

t6 = t[((t['Brick']==21)&(t['field']==10))|((t['Brick']==21)&(t['field']==8))|((t['Brick']==20)&(t['field']==5))
      |((t['Brick']==18)&(t['field']==2))|((t['Brick']==21)&(t['field']==11))|((t['Brick']==18)&(t['field']==8))
      |((t['Brick']==21)&(t['field']==7))|((t['Brick']==22)&(t['field']==18))|((t['Brick']==18)&(t['field']==14))
      ]

t7 = t[((t['Brick']==21)&(t['field']==12))|((t['Brick']==20)&(t['field']==16))|((t['Brick']==21)&(t['field']==3))
      |((t['Brick']==21)&(t['field']==2))|((t['Brick']==22)&(t['field']==12))|((t['Brick']==20)&(t['field']==10))
      |((t['Brick']==21)&(t['field']==1))|((t['Brick']==21)&(t['field']==4))|((t['Brick']==20)&(t['field']==4))
      |((t['Brick']==22)&(t['field']==17))|((t['Brick']==22)&(t['field']==6))]

t8 = t[((t['Brick']==21)&(t['field']==5))|((t['Brick']==18)&(t['field']==1))|((t['Brick']==22)&(t['field']==11))
      |((t['Brick']==23)&(t['field']==17))|((t['Brick']==23)&(t['field']==18))|((t['Brick']==18)&(t['field']==7))
      |((t['Brick']==23)&(t['field']==16))|((t['Brick']==21)&(t['field']==6))|((t['Brick']==22)&(t['field']==5))
      |((t['Brick']==18)&(t['field']==13))|((t['Brick']==20)&(t['field']==9))|((t['Brick']==20)&(t['field']==3))
      |((t['Brick']==20)&(t['field']==15))]

t9 = t[((t['Brick']==23)&(t['field']==15))|((t['Brick']==22)&(t['field']==16))|((t['Brick']==22)&(t['field']==10))
      |((t['Brick']==23)&(t['field']==11))|((t['Brick']==23)&(t['field']==10))|((t['Brick']==23)&(t['field']==12))
      |((t['Brick']==22)&(t['field']==4))|((t['Brick']==23)&(t['field']==14))|((t['Brick']==23)&(t['field']==9))
      |((t['Brick']==23)&(t['field']==8))|((t['Brick']==22)&(t['field']==15))|((t['Brick']==20)&(t['field']==2))
      |((t['Brick']==23)&(t['field']==4))|((t['Brick']==20)&(t['field']==8))|((t['Brick']==23)&(t['field']==13))
      |((t['Brick']==22)&(t['field']==9))]

t10 = t[((t['Brick']==23)&(t['field']==5))|((t['Brick']==23)&(t['field']==3))|((t['Brick']==20)&(t['field']==14))
      |((t['Brick']==22)&(t['field']==3))|((t['Brick']==23)&(t['field']==6))|((t['Brick']==23)&(t['field']==7))
      |((t['Brick']==23)&(t['field']==2))|((t['Brick']==23)&(t['field']==1))|((t['Brick']==22)&(t['field']==14))
      |((t['Brick']==20)&(t['field']==1))|((t['Brick']==22)&(t['field']==2))|((t['Brick']==20)&(t['field']==7))
      |((t['Brick']==20)&(t['field']==13))|((t['Brick']==22)&(t['field']==1))|((t['Brick']==22)&(t['field']==7))
      |((t['Brick']==22)&(t['field']==13))]

#### Save Files

In [ ]:
# tables containing JAGB stars for each region
jagb_regions = [r1, r2, r3, r4, r5, r6, r7, r8, r9, r10]
for i in range(10):
    jagb_regions[i].write('/Users/abigaillee/Photometry/M31 PHAT Photometry/Red uncor/JAGB stars/'+str(i)+'.fits',overwrite=True)

# tables containing all stars for each region
all_regions = [t1, t2, t3, t4, t5, t6, t7, t8, t9, t10]
for i in range(10):
    all_regions[i].write('/Users/abigaillee/Photometry/M31 PHAT Photometry/Red uncor/All/'+str(i)+'.fits',overwrite=True)

    